<a href="https://colab.research.google.com/github/Alina-89/Academic_RAG/blob/main/Academic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pymupdf

In [ ]:
import os
import gradio as gr
# Create the "Documents" folder
os.makedirs("Academic_docs", exist_ok=True)

In [ ]:
with gr.Blocks(css="""
/* 🌙 Dark Mode Background */
.gradio-container {
    background-color: #18181B;
    color: #FFFFFF;
    font-family: 'Poppins', sans-serif;
    padding: 20px;
    display: flex;
    justify-content: center;
}

/* Stack content vertically and center */
#main-column {
    display: flex;
    flex-direction: column;
    align-items: center;
    max-width: 800px;
    width: 100%;
    margin: auto;
}

/* 🖼️ Image Styling */
.gr-image {
    border-radius: 12px;
    box-shadow: 4px 4px 10px rgba(255, 255, 255, 0.2);
}

/* ✏️ Textbox Enhancements */
.gr-textbox {
    width: 90%;
    font-size: 18px;
    padding: 10px;
    border: 2px solid #4A4A4D;
}

/* 🎨 Button Customization */
.gr-button {
    background-color: #5A67D8;
    color: pink;
    font-size: 16px;
    padding: 12px 18px;
    border-radius: 8px;
    transition: 0.2s ease-in-out;
}

/* ✨ Refine Labels */
label {
    font-weight: bold;
    color: #D1D5DB;
}
""") as demo:

    with gr.Column(elem_id="main-column"):
        gr.Markdown("<h2 style='color: #EAB308;'>✏️ Create your summary</h2><p style='color: #9CA3AF;'>Upload your document</p>")

        file_input = gr.File(label="Upload a file")
        style_input = gr.Radio(["Academic", "Robotic", "Explain like I am 5"], label="Choose a style")
        generate_btn = gr.Button("Generate")

        summary_output = gr.Textbox(label="Generated Summary")


        generate_btn.click(inputs=[file_input, style_input],
                           outputs=[summary_output])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://753a5130802c610e41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
